In [6]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
from pyproj import Transformer

In [ ]:
# Construct baseline locations dataset
baseline_locations = {
    "Birmingham",
    "Westminster",
    "Leeds",
    "Glasgow City",
    "Sheffield",
    "City of Edinburgh",
    "Manchester",
    "City of London",
    "Tower Hamlets",
    "Liverpool",
}
baseline_locations = {
    "Birmingham",
    "City of Edinburgh",
    "City of London"
}

locations_path = "Data/Subnational_electricity_consumption_statistics_2005-2022.xlsx"
xls = pd.ExcelFile(locations_path)
sheet_names = [sheet for sheet in xls.sheet_names if sheet != "Cover sheet"]
baseline_data = []
for sheet in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, skiprows=4)
    df = df[['Local authority', 'Total consumption\n(GWh):\nAll meters']]
    df = df.rename(columns={
        'Total consumption\n(GWh):\nAll meters': "Total consumption"
    })
    filtered_df = df.loc[df['Local authority'].isin(baseline_locations)].copy()
    filtered_df['Year'] = sheet
    baseline_data.append(filtered_df)
baseline_df = pd.concat(baseline_data, ignore_index=True)

city_coordinates = {
    "Birmingham": {"latitude": 52.4862, "longitude": -1.8904},
    "Westminster": {"latitude": 51.500989, "longitude": -0.126118},
    "Leeds": {"latitude": 53.8008, "longitude": -1.5491},
    "Glasgow City": {"latitude": 55.8642, "longitude": -4.2518},
    "Sheffield": {"latitude": 53.3833, "longitude": -1.4667},
    "City of Edinburgh": {"latitude": 55.9533, "longitude": -3.1883},
    "Manchester": {"latitude": 53.4808, "longitude": -2.2426},
    "City of London": {"latitude": 51.5128, "longitude": -0.0918},
    "Tower Hamlets": {"latitude": 51.5118, "longitude": -0.0392},
    "Liverpool": {"latitude": 53.4084, "longitude": -2.9916},
}
city_coordinates = {
    "Birmingham": {"latitude": 52.4862, "longitude": -1.8904},
    "City of Edinburgh": {"latitude": 55.9533, "longitude": -3.1883},
    "City of London": {"latitude": 51.5128, "longitude": -0.0918},
}
coordinates_df = pd.DataFrame.from_dict(city_coordinates, orient='index').reset_index()
coordinates_df.rename(columns={'index': 'Local authority'}, inplace=True)
baseline_df = pd.merge(baseline_df, coordinates_df, on="Local authority", how="left")
baseline_df.to_csv("Data/baseline_demand.csv")

In [29]:
#Visualising the locations of these dummy centres
geo_df = baseline_df[['Local authority', 'longitude', 'latitude']].drop_duplicates()
m = folium.Map(location=[55.5, -3], zoom_start=5)
for i, city in enumerate(geo_df["Local authority"]):
    folium.CircleMarker(
        [geo_df["latitude"][i], geo_df["longitude"][i]],
        radius=5,
        color='blue',
        fill=True,
        popup=city).add_to(m)
m

In [41]:
## Baseline supply dataset
capacity_factors = {
    "Wind Offshore": 0.3954,
    "Wind Onshore": 0.2464,
    "Shoreline Wave": 0.1365,
    "Solar Photovoltaics": 0.1015,
    "Large Hydro": 37.5,
    "Small Hydro": 32.3,
}

supply_df = pd.read_csv("Data/repd-q3-oct-2024.csv", encoding='latin1')
supply_df = supply_df[['Site Name', 'Technology Type', "Installed Capacity (MWelec)", "X-coordinate", "Y-coordinate"]]
supply_df = supply_df.rename(columns={'Installed Capacity (MWelec)': 'Capacity'})
supply_df = supply_df.loc[supply_df['Technology Type'].isin(capacity_factors.keys())]
supply_df['Capacity'] = pd.to_numeric(supply_df['Capacity'], errors="coerce")
supply_df['Capacity Factor'] = supply_df['Technology Type'].map(capacity_factors)
hours_per_year = 24*365
supply_df['Annual Output'] = supply_df['Capacity'] * supply_df['Capacity Factor'] * hours_per_year
supply_df['Annual Output'] = supply_df['Annual Output']*10**-3 #convert from MW to GW

def select_largest(group):
    n_largest_tech_select_dict = {
    'Large Hydro': 1,
    "Small Hydro": 1,
    'Solar Photovoltaics': 2,
    'Shoreline Wave': 1,
    'Wind Offshore': 1,
    'Wind Onshore': 2
    }
    tech = group["Technology Type"].iloc[0]
    n = n_largest_tech_select_dict[tech]
    return group.nlargest(n, "Capacity")

baseline_supply = supply_df.groupby("Technology Type", group_keys=False).apply(select_largest)
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)
baseline_supply[["Longitude", "Latitude"]] = baseline_supply.apply(
    lambda row: pd.Series(transformer.transform(row["X-coordinate"], row["Y-coordinate"])),
    axis=1
)
# chosen baseline locations
baseline_supply_sites = ["Glendoe Hydro Scheme", "Berwick Bank Offshore Wind Farm ", "Gate Burton - Solar & Energy Storage Park"]
baseline_supply = baseline_supply.loc[baseline_supply['Site Name'].isin(baseline_supply_sites)]
baseline_supply.loc[baseline_supply['Site Name']=='Glendoe Hydro Scheme', ['Latitude', 'Longitude']] = 50.8225, -0.1372
# baseline_supply['']
# baseline_supply.to_csv("Data/baseline_supply.csv", index=False)
baseline_supply

/var/folders/l5/rhh7z5c528b5hjwqy30m3vzr0000gn/T/ipykernel_35323/2859157881.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  baseline_supply = supply_df.groupby("Technology Type", group_keys=False).apply(select_largest)


,Site Name,Technology Type,Capacity,X-coordinate,Y-coordinate,Capacity Factor,Annual Output,Longitude,Latitude
347,Glendoe Hydro Scheme,Large Hydro,100.0,239200,809000,37.5000,32850.00000,-0.137200,50.822500
6898,Gate Burton - Solar & Energy Storage Park,Solar Photovoltaics,531.0,483828,382860,0.1015,472.13334,-0.742569,53.336016
6935,Berwick Bank Offshore Wind Farm,Wind Offshore,4100.0,432997,728012,0.3954,14201.18640,-1.466402,56.443139


In [35]:
#Visualising the locations of these dummy centres
geo_df = baseline_df[['Local authority', 'longitude', 'latitude']].drop_duplicates()
m = folium.Map(location=[55.5, -3], zoom_start=5)
for i, city in enumerate(geo_df["Local authority"]):
    folium.CircleMarker(
        [geo_df["latitude"][i], geo_df["longitude"][i]],
        radius=5,
        color='blue',
        fill=True,
        popup=city).add_to(m)

for _, row in baseline_supply.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=4,  # Size of the marker
        color="red",  # Border color
        fill=True,
        fill_color="red",  # Fill color
        fill_opacity=0.8,
        popup=f"{row['Site Name']} ({row['Technology Type']})"
    ).add_to(m)
m

In [37]:
baseline_df

,Local authority,Total consumption,Year,latitude,longitude
0,Liverpool,2257.950350,2005,53.408400,-2.991600
1,Manchester,2686.109493,2005,53.480800,-2.242600
2,Leeds,3901.704721,2005,53.800800,-1.549100
3,Sheffield,2750.999300,2005,53.383300,-1.466700
4,Birmingham,4973.998311,2005,52.486200,-1.890400
...,...,...,...,...,...
171,City of London,1647.238759,2022,51.512800,-0.091800
172,Tower Hamlets,2416.880376,2022,51.511800,-0.039200
173,Westminster,3174.667400,2022,51.500989,-0.126118
174,City of Edinburgh,2021.536055,2022,55.953300,-3.188300
